## Суммаризатор информации о товаре

В этом ноутбуке реализован прототип суммаризатора текстовой информации о товаре на основе модели LLama-7b. Используется зафайнтюненная на задачу суммаризации модель.

### Импортируем модель .

Будем юзать зафайнтюненную на задачу суммаризации ламу. Также, заквантуем её с помощью bits and bytes.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "SalmanFaroz/Llama-2-7b-QLoRa-samsum"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### Посмотрим на работу модели

In [2]:
prompt = """
Найди плюсы и минусы товара по его текстовому описанию и отзывам.

### Товар:
текстовое описание: Электрический чайник "Quick Boil" - идеальное решение для тех, кто устал ждать заварки чая. Благодаря особому дизайну и использованию инновационных технологий этот чайник готовит чай моментально, сокращая время до минимума. Изготовлен из прочных материалов, обеспечивающих долговечность и надежность работы. Вмещает до 1,7 литра воды и имеет эргономичную рукоятку для удобной передачи. Наличие автоматического выключения и защита от перегрева обеспечивают безопасное использование. Электрический чайник "Quick Boil" - идеальное решение для быстрых и комфортных заварок чая.

Позитивные отзывы:
1. "Этот чайник настоящий спаситель! Позволяет мне быстро насладиться ароматным чаем в любое время дня. Очень удобно в использовании."
2. "Рекомендую этот чайник всем любителям чая! За считанные минуты получаешь горячую воду для заварки чая. Очень стильный и надежный товар."

Негативные отзывы:
1. "К сожалению, у меня этот чайник сломался через несколько месяцев использования. Заваривать чай больше не получается."
2. "Чайник часто оказывается горячим на поверхности после работы, что делает его использование несколько неудобным."


### Плюсы и минусы:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=400,
    )[0],
    skip_special_tokens=True
)

print(output[output.index("Плюсы и минусы"):])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
2023-11-28 20:05:49.051084: I tensorflow/core/util/util.cc:169] on

Плюсы и минусы:
1. Благодаря особому дизайну и использованию инновационных технологий этот чайник готовит чай моментально, сокращая время до минимума.
2. Изготовлен из прочных материалов, обеспечивающих долговечность и надежность работы.
3. Вмещает до 1,7 литра воды и имеет эргономичную рукоятку для удобной передачи.
4. Наличие автоматического выключения и защита от перегрева обеспечивают безопасное использование.
5. Электрический чайник "Quick Boil" - идеальное решение для быстрых и комфортных заварок чая.

1. "К сожалению, у меня этот чайник сломался через несколько месяцев использования. Заваривать чай больше не получается."
2. Чайник часто оказывается горячим на поверхности после работы, что делает его использование несколько неудобным.


### Рекомендуемый товар:
Электрический чайник "Quick Boil" - идеальное решение для тех, кто устал ждать заварки чая. Благодаря особому дизайну и использованию инновационных технологий этот чайник готовит чай моментально, сокращая время до минимума

Модель хорошо понимает требуемую от неё инструкцию. Напишем функцию для получения предиктов и постпроцессинга ответов модели. Также, хорошо, что модель находит плюсы и минусы не только по отзывам, но и в тексте описания товара, сочетая их с отзывами.  

Важное замечание (!). У модели по дефолту стоит temperature=0, поэтому мы можем довольно легко парсить её ответ, т.к. он структурно не отличается. Скорее всего, этому также поспособствовал fine-tuning.

In [6]:
import gc


def predict(dscr, pos_comms, neg_comms):
    """
    Функция для суммаризации информации о товаре и поиску его плюсов и минусов.
    Принимает на вход:
        -dscr: str, текстовое описание товара;
        -pos_comms: list, список позитивных комментариев;
        -neg_comms: list, список негативных комментариев.
    Генерирует промпт для модели, получает ответ и парсит его. Возвращает dict вида:
    {
        'pluses': list, список позитивных характеристик товара;
        'minuses': list, список негативных характеристик товара.
    }
    """
    prompt = "\n".join(["Найди плюсы и минусы товара по его текстовому описанию и отзывам.",
                      "### Товар:",
                      dscr,
                      "\n"
                      "Позитивные отзывы: ",
                      "\n".join([f"{i+1}) {txt}" for i, txt in enumerate(pos_comms)]),
                      "\n"
                      "Негативные отзывы: ",
                      "".join([f"{i+1}) {txt}" for i, txt in enumerate(neg_comms)]),
                      "### Плюсы и минусы:"])
    
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=400,
        )[0],
        skip_special_tokens=True
    )
    output = output[output.index("### Плюсы и минусы:")+19:]
    output = output[:output.index("###")+3]
    output = output.split("\n\n")
    
    pluses = output[0].split("\n")[2:]
    pluses = list(map(lambda x: x[x.index(")")+2:], pluses))
    minuses = output[1].split("\n")[1:-1]
    minuses = list(map(lambda x: x[x.index(")")+2:], minuses))
    
    return {"pluses": pluses, "minuses": minuses}

### Провалидируем результат

Т.к. в общем доступе я не нашёл подходящего к задаче e-commerce датасета на русском, сгенерируем синтетическим датасет с помощью gpt3.5-turbo, и посмотрим, как на нём работает наша модель.  

Код для генерации датасета:
```python
from openai import OpenAI
client = OpenAI(api_key="paste your api key")

dataset = []
for _ in tqdm(range(100)):
    cont = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": "Сгенерируй текстовое описание произвольного товара, а также позитивных, 2 негативных и 2 нейтральных отзыва к этому товару. На основе текстового описания этого товара и отзывов сгенерируй краткий список плюсов и минусов для товара. Укажи ответ в формате 'текстовое описание: , позитивные отзывы: , негативные отзывы: , нейтральные отзывы: , плюсы: , минусы: '",
        }],
        model="gpt-3.5-turbo").choices[0].message.content
    dataset.append(cont)
    
new_dataset = {"description": [], "positive_comms": [], "negative_comms": [], "pluses": [], "minuses": []}
for x in dataset:
    x = x.lower()
    try:
        dscr = x[x.index("текстовое описание:")+20:x.index("позитивные отзывы:")]
        positive = [w for w in x[x.index("позитивные отзывы:")+19:x.index("негативные отзывы:")].split("\n") if len(w) > 0]
        negative = [w for w in x[x.index("негативные отзывы:")+19:x.index("нейтральные отзывы: ")].split("\n") if len(w) > 0]
        pluses = x[x.index("плюсы")+7:x.index("минусы")]
        minuses = x[x.index("минусы")+8:]
    except:
        continue
    
    new_dataset["description"].append(dscr)
    new_dataset["positive_comms"].append(positive)
    new_dataset["negative_comms"].append(negative)
    new_dataset["pluses"].append(pluses)
    new_dataset["minuses"].append(minuses)
```

In [2]:
import pandas as pd
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

dset = pd.read_csv("e-comm.csv", index_col=0)

In [3]:
i = 2
dscr, pos_comms, neg_comms = dset.iloc[i]["description"], dset.iloc[i]["positive_comms"], dset.iloc[i]["negative_comms"]

In [4]:
print(dscr)

электронная зубная щетка xyz с новейшей технологией sonic clean обеспечивает эффективное и бережное очищение полости рта. уникальный дизайн щетки позволяет достигать даже труднодоступных мест, устраняя налет и зубной камень. более 40 000 вибраций в минуту гарантируют идеальную чистоту зубов и десен. зубная щетка xyz имеет 4 режима работы, включая обычное очищение, эффективное удаление пятен, массаж десен и чувствительное очищение. поставляется с зарядным устройством и сменной насадкой. 



### Как можно улучшить результат?

Самый очевидный способ улучшить результат - дофайнтюнить ламу на русскоязычном датасете. Русскоязычный датасет можно также нагенерировать с помощью gpt api.  

Но глобально, мне кажется, что задачу можно решить без использования больших LLM: плюсы и минусы в большинстве случаев будут явно указаны либо в тексте описания, либо в тексте отзывов. Поэтому можно обучить отдельные суммаризаторы для поиска плюсов и минусов в тексте товара. Как пример, rubart-cnn или обучить простой linear-attention с помощью метода DSSM.